In [0]:
import pandas as pd

In [0]:
catalog = 'operations'
schema = 'finance_staging'

In [0]:
def list_all_files(path):
    all_files = []
    try:
        items = dbutils.fs.ls(path)
        for item in items:
            if item.isDir():
                # Recursively call the function on subdirectories
                all_files.extend(list_all_files(item.path))
            else:
                # Add file path to the list
                all_files.append(item.path)
    except Exception as e:
        print(f"Error listing {path}: {e}")
    return all_files

# Run the function on your volume path
all_files = list_all_files("/Volumes/operations/finance_staging/edgar_data")

# Display results
for f in all_files:
    print(f)


In [0]:
volume_base = "/Volumes/operations/finance_staging/edgar_data"

#eventually replace with a function to get years, quarters as available in the volumes location

years = ['2025']
quarters = ['q1','q2']

files = ['pre','num','sub','tag']

for year in years:
    for quarter in quarters:
        for file in files:
            filing_path = f"{volume_base}/{year}/{quarter}/{file}.txt"
            filing = pd.read_csv(f"{filing_path}", sep="\t", low_memory=False, keep_default_na=False)
            filing['source_file'] = filing_path
            filing = spark.createDataFrame(filing)

            display(filing)
            # (filing
            #  .write
            #  .mode("overwrite")
            #  .option("overwriteSchema", "true")
            #  .saveAsTable(f'{catalog}.{schema}.raw_{year}_{quarter}_{file}_tbl'))